# 01 Datenbeschaffung der Social Media Posts von Donald Trump

Die nötigen Daten zur Analyse von Donald Trumps Social Media Posts müssen leider aus verschiedenen Quellen zusammen gestückelt werden:
- Trump-Twitter-Archive (2009-2021)
- TTA, aber händischer Download von immer jeweils 2000 Posts (2021-2024)
- Scraping mit Playwright für die neuesten & aktuellsten Daten (2024-2025)

## Teil 1: Scraping von Daten 2024-2025

### Scraping mit Playwright

In [1]:
## zur Installation der verwendeten Pakete:
# !pip install playwright pandas aiohttp aiofiles os ssl certifi
# einfach die Zeile mit !pip auskommentieren und durchführen; danach Kernel neu laden
# playwright install ##in shell ausführen

### Wie sollen die Daten am Ende aussehen?
- Id: Nummer des Posts
- author: Donald Trump @realdonaldtrump
- platform: Truthsocial or X (Twitter)
- date: ganzes Datum (ohne Uhrzeit)
- day: Tag des Posts
- month: Monat des Posts
- year: Jahr des Posts
- time: Uhrzeit des Posts
- text: ganzer Text (ohne Datum, Uhrzeit, Autor und Plattform)
- image : image_path -> der Weg zu den Bildern wird lokal gespeichert.

### Wie sieht die Webseite aus:

- Quellcode auf Website anschauen: https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc
- Seite lädt Inhalte interaktiv mit Java-Script nach (nicht statisch)
- alle Posts sind in jeweils einzelnen Blöcken gespeichert
- im jeweiligen Block ist einmal das Bild gespeichert und zudem DAutor, Plattform, Datum, Uhrzeit und Text in einem gemeinsamen Block
- Suchmaske auf der Website implementiert
- tägliche Ergänzung neuer Posts
- Blick auf die URL: Beim Scrollen ändert sich die Seitenzahl in der URL
- zum 1.August waren es 87.640 Posts (X und Truthsocial)
- wahrscheinlich circa 5.000 Seiten

### Wahl des Tools:
- Beautifulsoup: schon älter, nur für statische Websites geeignet, braucht länger
- Selectolax: modern und deutlich schneller als Beautiful, wird außerdem seltener blockiert, allerdings ebenfalls nur für statische Seiten
- Selenium: dynamische Alternative, gute Möglichkeit
- Playwright: relativ modern, sehr schnell und effizient, für dynamische Seiten geeignet

### Wie viele Posts gibt es überhaupt auf der Webseite?
- Auf der Webseite steht, dass es 87.656 gibt (Stand 6.8.2025)
- Heruntergeladen werden nur die neueren Posts (November 2024-August 2025), als Ergänzung zum TTA

#### Probleme und Lösungen:
- die Seite lädt dynamisch nach: Playwright verwenden für dynamische Webseiten
- dynamisches Nachladen: wie komme ich zum Ende der Posts?
- tausend doppelte und dreifache Posts: durch Seiten durchiterieren bringt leider nur doppelte Posts
- doppelte Posts: Key erstellen, mit dem abgeglichen werden kann
- dynamisches Nachladen der Seiten: statt durch Seiten iterieren lieber Scrollen!
- Seiten laden langsam nach: sleep einbauen
- durch das Herunterladen der Bilder: Programm wird sehr langsam :(, daher parallele Worker etablieren & asynchrone Methoden, statt synchron
- Programm stürzt ab, bzw findet keine nicht immer neue Posts, weil die Seiten langsam nachladen: länger warten (sleep(3)) und Posts direkt in csv speichern und nach einem Neustart bereits Gespeichertes aus dem File laden

#### Code für das Scraping mit Playwright

... zuerst die inhalte der Posts:

In [1]:
### Test für die einzelnen Posts ###
import re
from datetime import datetime

def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 2:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        start_idx = 2
        if len(lines) > 2 and lines[2].startswith("View"):  # z.B. "View on ..."
            start_idx = 3
        content_lines = lines[start_idx:]

    content_text = " ".join(content_lines).strip()

    content_text = re.sub(r"\s{2,}", " ", content_text)

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": content_text
        }
    except Exception:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": content_text
        }

# Testfunktion
def test_extract_metadata():
    sample_post = """
    Donald Trump
    @realDonaldTrump • Twitter • January 6, 2021 @ 3:45 PM
    View on Twitter
    This is a test post
    with multiple lines
    and even more text.
    
    As obviously, this is also part of the text.
    
    And what about this? @you
    Look here!
    """

    result = extract_metadata_text(sample_post)
    print("Autor:", result["author"])
    print("Plattform:", result["platform"])
    print("Datum:", result["date"])
    print("Zeit:", result["time"])
    print("Text:", result["text"])

# Testlauf
if __name__ == "__main__":
    test_extract_metadata()


Autor: Donald Trump @realDonaldTrump
Plattform: Twitter
Datum: 2021-01-06
Zeit: 15:45
Text: This is a test post with multiple lines and even more text. As obviously, this is also part of the text. And what about this? @you Look here!


Folgender Code lädt 7.176 Posts herunter (vom 2.11.2024 bis zum 25.08.2025):

In [2]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl
import hashlib
import aiofiles

nest_asyncio.apply()
os.makedirs("images", exist_ok=True)

sslcontext = ssl.create_default_context(cafile=certifi.where())
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

CSV_FILE = "factbase_posts_clean.csv"

# Anzahl paralleler Download-Worker
num_workers = 10

async def download_worker(queue, session):
    """Paralleles Herunterladen der Daten (vor allem der Bilder),
        damit das Programm schneller läuft"""
    os.makedirs("images", exist_ok=True)
    while True:
        item = await queue.get()
        if item is None: 
            queue.task_done()
            break

        image_url, post, filename = item
        try:
            async with session.get(image_url, ssl=sslcontext) as resp:
                if resp.status == 200:
                    fpath = os.path.join("images", filename)
                    async with aiofiles.open(fpath, "wb") as f:
                        await f.write(await resp.read())
                    post["image_path"] = fpath
        except Exception as e:
            print(f"Fehler beim Download {image_url}: {e}")
        finally:
            queue.task_done()
            

             
def extract_metadata_text(text: str):
    """Extrahieren der Metadaten, die auf der Webseite in einem Block angezeigt werden"""
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 2:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        start_idx = 2
        if len(lines) > 2 and lines[2].startswith("View"):  # z.B. "View on ..."
            start_idx = 3
        content_lines = lines[start_idx:]

    # Absatzmarker durch Leerzeichen ersetzen und doppelte Leerzeichen anpassen
    content_text = " ".join(content_lines).strip()
    content_text = re.sub(r"\s{2,}", " ", content_text)

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": content_text
        }
    except Exception:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": content_text
        }
    
def make_post_key(data, include_image=False, include_text=True):
    """
    Generiert einen eindeutigen Schlüssel für jeden Post, um Duplikate zu vermeiden.
    Text und Bild sind optional; nur Posts ohne jegliche Metadaten UND ohne Text UND ohne Bild werden verworfen
    """
    # Metadaten prüfen
    #author = str(data.get("author", "")).strip() 
    #Autor ist eigentlich immer identisch, daher für Key sinnlos
    platform = str(data.get("platform", "")).strip()
    date = str(data.get("date", "")).strip()
    time = str(data.get("time", "")).strip()
    #text = str(data.get("text", "")).strip()
    img = data.get("image_path")
    
     # Wenn gar nichts da ist: kein valider Key
    if not (platform or date or time or img): #Text kann auch leer sein, daher weglassen
        return None
    
    parts = []
    parts.extend([platform, date, time]) #Basisdaten

    # Text (optional)
    if include_text:
        text_val = str(data.get("text", "")).strip().lower()
        text_norm = re.sub(r"\s+", " ", text_val)
        parts.append(text_norm)
    
    # Bild (optional)
    if include_image and isinstance(img, str) and img.strip():
        parts.append(os.path.basename(img.strip()))

    raw_key = "|".join(parts).strip()
    if not raw_key:
        return None

    return hashlib.md5(raw_key.encode("utf-8")).hexdigest()


async def scrape_all_dynamic(max_posts=90000, max_no_new=5):
    """Scraping der Daten inclusive Checker, ob schon ein File mit Daten vorhanden ist, 
        um an der Stelle weiter arbeiten zu können, wenn das Programm abbricht.
        Bilddownload, Scrollfunktion und Worker beenden."""
    posts_data = []
    seen_posts = set()

    # Fortsetzung, damit mit bereits vorhandenen Daten weiter gearbeitet werden kann
    if os.path.exists(CSV_FILE):
        print(f"Vorhandene Datei gefunden: {CSV_FILE} – Lade gespeicherte Posts...")
        df_existing = pd.read_csv(CSV_FILE)
        posts_data = df_existing.to_dict("records")
        seen_posts = {make_post_key(row) for row in posts_data}
        print(f"{len(posts_data)} Posts bereits geladen – setze fort...")

    no_new_rounds = 0

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        await page.goto("https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc")
        await asyncio.sleep(2)

        queue = asyncio.Queue()
        workers = [asyncio.create_task(download_worker(queue, session)) for _ in range(num_workers)]

        while True:
            try:
                await page.wait_for_selector("div.block", timeout=30000)
                blocks = await page.query_selector_all("div.block")
            except:
                print("Keine weiteren Posts, breche ab.")
                break

            print(f"Aktuell {len(posts_data)} Posts gespeichert – {len(blocks)} Blöcke auf der Seite sichtbar")

            new_count = 0
            for block in blocks:
                if len(posts_data) >= max_posts:
                    break

                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Basis-Post
                    post = {
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": None,
                        "image_url": None
                    }

                    # Bild-Download vorbereiten
                    img_src = None
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                img_src = src
                                filename = f"{hashlib.md5(src.encode()).hexdigest()}.jpg"
                                await queue.put((src, post, filename))
                    except:
                        pass

                    post["image_url"] = img_src
                    
                    # Schlüssel generieren
                    key = make_post_key(post, include_image=False, include_text=True)
                    if not key or key in seen_posts:
                        continue
                    seen_posts.add(key)
                    
                    posts_data.append(post)
                    new_count += 1

                except Exception as e:
                    print(f"Fehler bei Post: {e}")

            print(f"Neu hinzugekommen: {new_count} Posts")

            # Scrollen und no_new_rounds prüfen
            if new_count == 0:
                no_new_rounds += 1
                print(f"Keine neuen Posts ({no_new_rounds}/{max_no_new})")
                if no_new_rounds >= max_no_new:
                    break
            else:
                no_new_rounds = 0

            # Scrollen nur, wenn max_posts noch nicht erreicht wurde
            if len(posts_data) < max_posts:
                last_height = await page.evaluate("document.body.scrollHeight")
                await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
                await asyncio.sleep(3)
                new_height = await page.evaluate("document.body.scrollHeight")
                if new_height == last_height:
                    print(f"Scrollen brachte nichts Neues ({no_new_rounds}/{max_no_new})")
                    break
            else:
                print(f"Maximale Anzahl {max_posts} erreicht.")
                break

        # Queue abwarten
        await queue.join()
        for _ in range(num_workers):
            await queue.put(None)
        await asyncio.gather(*workers)

        await browser.close()

    # IDs vergeben und CSV speichern
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    fff = pd.DataFrame(posts_data)
    cols = ["id", "author", "platform", "date", "time", "day", "month", "year", "text", "image_path"]
    fff = fff[cols]
    fff.to_csv(CSV_FILE, index=False, encoding="utf-8")
    print(f"Scraping abgeschlossen. Gesamt: {len(fff)} Posts.")


# Wie viele Posts?
    total = len(fff)
    print("===================================")
    print(f"Gesamt:     {total}")
    print("===================================")

await scrape_all_dynamic(max_posts=90000)
# Hinweis: Der Code wurde durch Fehleranalysen von den ChatGPT Versionen 3.5, 4 und 5 verbessert

Vorhandene Datei gefunden: factbase_posts_clean.csv – Lade gespeicherte Posts...
7161 Posts bereits geladen – setze fort...
Aktuell 7161 Posts gespeichert – 53 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 11 Posts
Aktuell 7172 Posts gespeichert – 51 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 0 Posts
Keine neuen Posts (1/5)
Aktuell 7172 Posts gespeichert – 101 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 4 Posts
Scrollen brachte nichts Neues (0/5)
Scraping abgeschlossen. Gesamt: 7176 Posts.
Gesamt:     7176


In [19]:
# Wie sehen die Daten aus?
import pandas as pd
ppp = pd.read_csv("factbase_posts_clean.csv")
print(ppp.tail(10).T)

                                                         5180  \
id                                                       5860   
author                          Donald Trump @realDonaldTrump   
platform                                         Truth Social   
date                                               2025-08-24   
time                                                    21:00   
day                                                      24.0   
month                                                  August   
year                                                   2025.0   
text        I have a Constitutional Right to appoint Judge...   
image_path        images/aa7ffa6016129bdf398b5e58b43950f3.jpg   

                                                         5181  \
id                                                       5861   
author                          Donald Trump @realDonaldTrump   
platform                              Deleted •  Truth Social   
date                    

In [20]:
ppp.shape

(5190, 10)

In [21]:
ppp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5190 entries, 0 to 5189
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          5190 non-null   int64  
 1   author      5190 non-null   object 
 2   platform    4886 non-null   object 
 3   date        5190 non-null   object 
 4   time        5190 non-null   object 
 5   day         5190 non-null   float64
 6   month       5190 non-null   object 
 7   year        5190 non-null   float64
 8   text        3591 non-null   object 
 9   image_path  1423 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 405.6+ KB


In [22]:
oldest_30 = ppp.sort_values('date', ascending=True).head(30)
print(oldest_30)

        id                         author                 platform  \
5175  5855  Donald Trump @realDonaldTrump                      NaN   
5174  5854  Donald Trump @realDonaldTrump                      NaN   
5176  5856  Donald Trump @realDonaldTrump             Truth Social   
5173  5853  Donald Trump @realDonaldTrump             Truth Social   
5171  5851  Donald Trump @realDonaldTrump             Truth Social   
5142  5822  Donald Trump @realDonaldTrump                      NaN   
5141  5821  Donald Trump @realDonaldTrump                      NaN   
5140  5820  Donald Trump @realDonaldTrump                      NaN   
5139  5819  Donald Trump @realDonaldTrump                      NaN   
5138  5818  Donald Trump @realDonaldTrump                      NaN   
5137  5817  Donald Trump @realDonaldTrump             Truth Social   
5136  5816  Donald Trump @realDonaldTrump             Truth Social   
5135  5815  Donald Trump @realDonaldTrump             Truth Social   
5134  5814  Donald T

In [23]:
newest_30 = ppp.sort_values('date', ascending=False).head(30)
print(newest_30)

        id                         author                 platform  \
5189  7168  Donald Trump @realDonaldTrump             Truth Social   
5188  7167  Donald Trump @realDonaldTrump             Truth Social   
5187  7166  Donald Trump @realDonaldTrump             Truth Social   
5186  7165  Donald Trump @realDonaldTrump             Truth Social   
5185  7164  Donald Trump @realDonaldTrump             Truth Social   
5184  7163  Donald Trump @realDonaldTrump             Truth Social   
5183  7162  Donald Trump @realDonaldTrump             Truth Social   
1        2  Donald Trump @realDonaldTrump             Truth Social   
5182  5862  Donald Trump @realDonaldTrump             Truth Social   
5181  5861  Donald Trump @realDonaldTrump  Deleted •  Truth Social   
5180  5860  Donald Trump @realDonaldTrump             Truth Social   
5179  5859  Donald Trump @realDonaldTrump             Truth Social   
5178  5858  Donald Trump @realDonaldTrump             Truth Social   
5177  5857  Donald T

In [24]:
ppp.text[9]

nan

In [25]:
# Wie viele Posts enthalten keine Angabe zur Zeit?
count_without_time = ppp["time"].isna().sum()
print(f"Anzahl der Posts ohne Zeitangabe: {count_without_time}")

Anzahl der Posts ohne Zeitangabe: 0


In [26]:
# Wie viele Posts enthalten keine Angabe zum Datum?
count_without_day = ppp["day"].isna().sum()
print(f"Anzahl der Posts ohne Tag: {count_without_day}")

Anzahl der Posts ohne Tag: 0


In [27]:
# Alle Zeilen anzeigen, wo day oder year NaN ist
missing = ppp[ppp["day"].isna() | ppp["year"].isna()]
print(missing)
# Die beiden Zeilen wollen wir nicht haben, da sie kein Teil der Posts von Trump sind, 
#sondern nur auf der Webseite stehen.
# --- die beiden Zeilen sind leider nicht mehr sichtbar, da sie gelöscht wurden und dieser Code versehentlich erneut ausgeführt wurde ---

Empty DataFrame
Columns: [id, author, platform, date, time, day, month, year, text, image_path]
Index: []


In [28]:
bad_idx = missing.index
ppp = ppp.drop(index=bad_idx).reset_index(drop=True)
count_without_day = ppp["day"].isna().sum()
print(f"Anzahl der Posts ohne Tag: {count_without_day}")

Anzahl der Posts ohne Tag: 0


In [29]:
# Wie viele Posts enthalten keine Angabe zum Datum?
count_without_year = ppp["year"].isna().sum()
print(f"Anzahl der Posts ohne Jahr: {count_without_year}")

Anzahl der Posts ohne Jahr: 0


In [30]:
ppp_single = ppp.drop_duplicates(subset=['time','date', 'image_path'], keep="first")
print(f"Vorher: {len(ppp)} Zeilen, danach: {len(ppp_single)} Zeilen ohne Duplikate.")

Vorher: 5190 Zeilen, danach: 5190 Zeilen ohne Duplikate.


In [31]:
print(ppp_single.sort_values('date').head(5))

        id                         author      platform        date   time  \
5175  5855  Donald Trump @realDonaldTrump           NaN  2024-11-02  23:29   
5174  5854  Donald Trump @realDonaldTrump           NaN  2024-11-02  23:37   
5176  5856  Donald Trump @realDonaldTrump  Truth Social  2024-11-02  23:13   
5173  5853  Donald Trump @realDonaldTrump  Truth Social  2024-11-02  23:57   
5171  5851  Donald Trump @realDonaldTrump  Truth Social  2024-11-03  01:13   

      day     month    year  \
5175  2.0  November  2024.0   
5174  2.0  November  2024.0   
5176  2.0  November  2024.0   
5173  2.0  November  2024.0   
5171  3.0  November  2024.0   

                                                   text  \
5175  THANK YOU—GREENSBORO, NORTH CAROLINA! #MAGA ht...   
5174  Three beautiful MAGA RALLIES today in Gastonia...   
5176       RT @realDonaldTrump11/2/24 | SALEM, VIRGINIA   
5173                                                NaN   
5171  https://www.breitbart.com/clips/2009/10/05/

In [32]:
ppp = ppp.drop_duplicates(subset=['time','date', 'image_path'], keep="first")
print(len(ppp))

5190


In [33]:
ppp[ppp.duplicated(subset=["text"], keep=False)]

,id,author,platform,date,time,day,month,year,text,image_path
2,4,Donald Trump @realDonaldTrump,Truth Social,2025-08-24,12:22,24.0,August,2025.0,NaN,NaN
7,9,Donald Trump @realDonaldTrump,Truth Social,2025-08-23,16:57,23.0,August,2025.0,NaN,NaN
9,11,Donald Trump @realDonaldTrump,Truth Social,2025-08-22,20:33,22.0,August,2025.0,NaN,NaN
20,25,Donald Trump @realDonaldTrump,Truth Social,2025-08-22,09:10,22.0,August,2025.0,MAKE AMERICA GREAT AGAIN!,NaN
26,31,Donald Trump @realDonaldTrump,Truth Social,2025-08-21,22:33,21.0,August,2025.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5117,5797,Donald Trump @realDonaldTrump,Truth Social,2024-11-04,08:02,4.0,November,2024.0,NaN,images/4be05e52bf07d0a08528ec6d6d1a0a5d.jpg
5143,5823,Donald Trump @realDonaldTrump,Truth Social,2024-11-03,17:46,3.0,November,2024.0,NaN,images/4f86e8bde090cef96ed07704138daf7f.jpg
5160,5840,Donald Trump @realDonaldTrump,Truth Social,2024-11-03,10:17,3.0,November,2024.0,NaN,images/13e62726b18ccf112b58656cea6e7f6a.jpg
5173,5853,Donald Trump @realDonaldTrump,Truth Social,2024-11-02,23:57,2.0,November,2024.0,NaN,images/39683b4916fca04a5a6f2ee720bacf73.jpg


In [34]:
texts_no_nan = ppp["text"].dropna()
counts = texts_no_nan.value_counts().reset_index()
counts.columns = ["text", "count"]
counts["count"] = counts["count"].astype(int)
duplicates = counts[counts["count"] > 1]
ppp_duplicates = ppp.merge(duplicates, on="text", how="inner")
display(ppp_duplicates)

,id,author,platform,date,time,day,month,year,text,image_path,count
0,25,Donald Trump @realDonaldTrump,Truth Social,2025-08-22,09:10,22.0,August,2025.0,MAKE AMERICA GREAT AGAIN!,NaN,9
1,59,Donald Trump @realDonaldTrump,Truth Social,2025-08-20,10:22,20.0,August,2025.0,https://www.whitehouse.gov/articles/2025/08/pr...,NaN,2
2,65,Donald Trump @realDonaldTrump,Truth Social,2025-08-20,08:37,20.0,August,2025.0,https://truthsocial.com/@mrddmia/1150491309019...,NaN,2
3,85,Donald Trump @realDonaldTrump,Truth Social,2025-08-19,14:35,19.0,August,2025.0,https://www.whitehouse.gov/articles/2025/08/pr...,NaN,2
4,95,Donald Trump @realDonaldTrump,Truth Social,2025-08-19,07:02,19.0,August,2025.0,https://truthsocial.com/@mrddmia/1150491309019...,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...
363,5599,Donald Trump @realDonaldTrump,Truth Social,2024-11-17,01:54,17.0,November,2024.0,https://nypost.com/2024/11/16/us-news/trump-gr...,images/84816480668089a0bb465973be74fc9d.jpg,2
364,5617,Donald Trump @realDonaldTrump,Truth Social,2024-11-13,22:10,13.0,November,2024.0,RT @realDonaldTrump,images/a5c1d0aca43b4d90250a32d03f961366.jpg,14
365,5743,Donald Trump @realDonaldTrump,NaN,2024-11-05,13:03,5.0,November,2024.0,Republicans: We are doing GREAT! Stay on Line....,images/49a35cd8cff0bd7bb7df11a0d864c0c5.jpg,2
366,5744,Donald Trump @realDonaldTrump,Truth Social,2024-11-05,13:01,5.0,November,2024.0,Republicans: We are doing GREAT! Stay on Line....,images/b633833ed1a44c1911e5bd19347f3a7d.jpg,2


In [35]:
ppp["text"].value_counts()[ppp["text"].value_counts() > 3]

text
RT @realDonaldTrump            14
MAKE AMERICA GREAT AGAIN!       9
MAKE AMERICA GREAT AGAIN!!!     7
Name: count, dtype: int64

In [36]:
# Bereinigung der Nan-Werte
ppp['text'] = ppp['text'].fillna("").str.strip()

In [37]:
ppp.to_csv("factbase_posts_clean.csv", index=False)

In [38]:
ppp.shape

(5190, 10)

In [39]:
# Reichen die gescrapten Daten weit genug? 
#Bis zum 04.11.2024 sind Daten des Trump-Twitter-Archivs vorhanden.
oldest_single_30 = ppp_single.sort_values('date').head(30)
print(oldest_single_30.T)

                                                         5175  \
id                                                       5855   
author                          Donald Trump @realDonaldTrump   
platform                                                  NaN   
date                                               2024-11-02   
time                                                    23:29   
day                                                       2.0   
month                                                November   
year                                                   2024.0   
text        THANK YOU—GREENSBORO, NORTH CAROLINA! #MAGA ht...   
image_path        images/00a8daa2f3c80102ee965af26597442d.jpg   

                                                         5174  \
id                                                       5854   
author                          Donald Trump @realDonaldTrump   
platform                                                  NaN   
date                    

In [40]:
count_without_id = ppp["id"].isna().sum()
print(f"Anzahl der Posts ohne ID: {count_without_id}")

Anzahl der Posts ohne ID: 0


In [41]:
post_1452 = ppp[ppp['time'] == '14:52']
print(post_1452)

        id                         author      platform        date   time  \
1008  1103  Donald Trump @realDonaldTrump  Truth Social  2025-06-26  14:52   
1816  2001  Donald Trump @realDonaldTrump  Truth Social  2025-05-04  14:52   
2562  2829  Donald Trump @realDonaldTrump  Truth Social  2025-03-16  14:52   
2858  3252  Donald Trump @realDonaldTrump           NaN  2025-02-24  14:52   
3706  4366  Donald Trump @realDonaldTrump  Truth Social  2024-12-17  14:52   
4408  5088  Donald Trump @realDonaldTrump           NaN  2025-02-24  14:52   

       day     month    year  \
1008  26.0      June  2025.0   
1816   4.0       May  2025.0   
2562  16.0     March  2025.0   
2858  24.0  February  2025.0   
3706  17.0  December  2024.0   
4408  24.0  February  2025.0   

                                                   text  \
1008  The Democrats are the ones who leaked the info...   
1816  Lee Zeldin: “At the Trump EPA, the status quo ...   
2562  I just won the Golf Club Championship, probab

##### Daten in json umwandeln:

In [42]:
import pandas as pd
ppp = pd.read_csv("factbase_posts_clean.csv")
ppp.to_json("factbase_posts_clean.json", orient="records", force_ascii=False, indent=2)
print(ppp.head())

   id                         author      platform        date   time   day  \
0   1  Donald Trump @realDonaldTrump  Truth Social  2025-08-24  12:24  24.0   
1   2  Donald Trump @realDonaldTrump  Truth Social  2025-08-24  12:23  24.0   
2   4  Donald Trump @realDonaldTrump  Truth Social  2025-08-24  12:22  24.0   
3   5  Donald Trump @realDonaldTrump  Truth Social  2025-08-24  10:16  24.0   
4   6  Donald Trump @realDonaldTrump  Truth Social  2025-08-24  10:12  24.0   

    month    year                                               text  \
0  August  2025.0  I played Golf yesterday with the Great Roger C...   
1  August  2025.0  https://humanevents.com/2025/08/21/shea-bradle...   
2  August  2025.0                                                NaN   
3  August  2025.0  Did Wes Moore, the Governor of Maryland, lie a...   
4  August  2025.0  Governor Wes Moore of Maryland has asked, in a...   

  image_path  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  


## Teil 2: Daten von 2009-2024
- sofort zum Download bereit: 2009-2021
- in 2.000er Schritten einzeln herunterladen: 2022-2024

#### Die restlichen Daten wurden von der Webseite https://www.thetrumparchive.com heruntergeladen.

In [43]:
# zuerst der File mit Daten von 2009-2021
import pandas as pd
fff = pd.read_csv("tweets_01-08-2021.csv")
print(fff.tail(10).T)

                                                       56561  \
id                                       1212166009446162432   
text       RT @heatherjones333: MAGNIFICENT TRUMP- KEEPIN...   
isRetweet                                                  t   
isDeleted                                                  f   
device                                    Twitter for iPhone   
favorites                                                  0   
retweets                                                6452   
date                                     2020-01-01 00:17:52   
isFlagged                                                  f   

                                                       56562  \
id                                       1212165377477750786   
text       RT @heatherjones333: 🔥🔥🔥🔥🔥Lindsey Graham: Trum...   
isRetweet                                                  t   
isDeleted                                                  f   
device                                 

In [44]:
fff.shape

(56571, 9)

Nun sollen alle restlichen json-files von 2021-2024, die ich einzeln herunter laden musste (mit jeweils 2000 Posts), zusammen gehängt werden:

In [51]:
import glob
files = glob.glob("TTA/*.json")
print(files)

['TTA/januarmarch24.json', 'TTA/marchmai24.json', 'TTA/juliseptember24.json', 'TTA/septembernovember23.json', 'TTA/apriljuli23.json', 'TTA/novemberjanuar24.json', 'TTA/spetemberdezember24.json', 'TTA/juliseptember23.json', 'TTA/oktoberjanuar22.json', 'TTA/januarapril23.json', 'TTA/junioktober22.json', 'TTA/maijuli24.json', 'TTA/tweets_01-08-2021.json', 'TTA/januar21juni22.json']


Die Dateien, die ich herunterladen musste, waren scheinbar keine sauberen json-files.
Sämtliche Steuerzeichen wie \n, \t, \r, Nullbytes oder ungewöhnliche Kontrollcodes werden durch ein Leerzeichen " " ersetzt, da JSON nur bestimmte Zeichen enthalten darf (z.B. ", \n, \t nur in Strings mit Escape). Wenn in den Dateien unescapte Steuerzeichen enthalten sind, bricht json.loads ab.

In [52]:
import json, pandas as pd, glob, re

dfs = []
for file in glob.glob("TTA/*.json"):
    with open(file, "r", encoding="utf-8") as f:
        raw = f.read()
        cleaned = re.sub(r"[\x00-\x1f\x7f]", " ", raw)
    for loader in (lambda x: json.loads(x), lambda x: [json.loads(line) for line in x.splitlines() if line.strip()]):
        try:
            data = loader(cleaned)
            if isinstance(data, dict): data = [data]
            dfs.append(pd.DataFrame(data))
            print(f"Bereinigt geladen: {file}")
            break
        except: 
            continue

if dfs:
    tta_full = pd.concat(dfs, ignore_index=True)
    print("Gesamtdaten:", tta_full.shape)
    print(tta_full.head())
    tta_full.to_csv("tta_full.csv", index=False, encoding="utf-8")
    tta_full.to_json("tta_full.json", orient="records", force_ascii=False, indent=2)
    print("Gespeichert als tta_full.csv und tta_full.json")
else:
    print("Keine gültigen JSON-Dateien geladen.")

Bereinigt geladen: TTA/januarmarch24.json
Bereinigt geladen: TTA/marchmai24.json
Bereinigt geladen: TTA/juliseptember24.json
Bereinigt geladen: TTA/septembernovember23.json
Bereinigt geladen: TTA/apriljuli23.json
Bereinigt geladen: TTA/novemberjanuar24.json
Bereinigt geladen: TTA/spetemberdezember24.json
Bereinigt geladen: TTA/juliseptember23.json
Bereinigt geladen: TTA/oktoberjanuar22.json
Bereinigt geladen: TTA/januarapril23.json
Bereinigt geladen: TTA/junioktober22.json
Bereinigt geladen: TTA/maijuli24.json
Bereinigt geladen: TTA/tweets_01-08-2021.json
Bereinigt geladen: TTA/januar21juni22.json
Gesamtdaten: (80358, 9)
            date favorites                  id isRetweet retweets  \
0  1709611886295     22410  112041124575579316     False     4357   
1  1709606786303     16477  112040790347547040     False     3420   
2  1709606689853     11978  112040784026627289     False     3468   
3  1709600790251     14970  112040397390384141     False     3429   
4  1709599014526     10254

In [53]:
tta_full.text[1]

'<p><span  class="quote-inline"><br/>RT:  https://truthsocial.com/users/realDonaldTrump/statuses/112037150280659739</span>THANK  YOU, NORTH DAKOTA! <a  href="https://links.truthsocial.com/link/110119864581902473"  rel="nofollow noopener noreferrer" target="_blank"><span  class="invisible">https://</span><span  class="">DonaldJTrump.com</span><span  class="invisible"></span></a></p>'

In [54]:
import pandas as pd
tta_full = pd.read_json("tta_full.json", orient="records")
print(tta_full.columns.tolist())

['date', 'favorites', 'id', 'isRetweet', 'retweets', 'text', 'isDeleted', 'device', 'isFlagged']


Das Datum soll getrennt werden, damit es besser zu den Daten von "factbase_posts_clean.json" passt.
Außerdem ist das Datum noch seltsam codiert:

In [55]:
### scheinbar gibt es eine spezifische Art für Twitter, das Datum darzustellen:
## als Twitter Snowflake (laut ChatGPT)
import pandas as pd
tta_full = pd.read_json("tta_full.json", orient="records")
print(tta_full['date'].head(5))
print(tta_full['date'].dtype)

0   1970-01-01 00:28:29.611886295
1   1970-01-01 00:28:29.606786303
2   1970-01-01 00:28:29.606689853
3   1970-01-01 00:28:29.600790251
4   1970-01-01 00:28:29.599014526
Name: date, dtype: datetime64[ns]
datetime64[ns]


In [56]:
tta_full['date'] = pd.to_datetime(tta_full['date'], unit='ms', errors='coerce')
print(tta_full['date'].dt.year.value_counts().sort_index())
# Die Datumsangaben stimmen nicht! 2022-2024 werden als 1970 gespeichert

date
1970    23787
2009       56
2010      142
2011      773
2012     3531
2013     8144
2014     5784
2015     7536
2016     4225
2017     2602
2018     3568
2019     7818
2020    12236
2021      156
Name: count, dtype: int64


Die Daten sind zusammengesetzt aus TTA 2009-2021, welche mit Twitter-Snowflake kodiert sind.
Die manuell heruntergeladenen Daten von TTA (immer 2.000 Tweets auf einmal) stammen größtenteils von Truth Social. Dadurch ist das Datum anders verschlüsselt als die Daten 2009-2021.
Mit folgendem Code wurden die Daten daher erneut zusammengeführt und korrigiert:

In [101]:
import pandas as pd
import json
from pathlib import Path

# Alle JSON-Dateien in einem DataFrame speichern
file_paths = [
    "TTA/januarmarch24.json",
    "TTA/marchmai24.json",
    "TTA/juliseptember24.json",
    "TTA/septembernovember23.json",
    "TTA/apriljuli23.json",
    "TTA/novemberjanuar24.json",
    "TTA/spetemberdezember24.json",
    "TTA/juliseptember23.json",
    "TTA/oktoberjanuar22.json",
    "TTA/januarapril23.json",
    "TTA/junioktober22.json",
    "TTA/maijuli24.json",
    "TTA/tweets_01-08-2021.json",
    "TTA/januar21juni22.json"
]

dfs = []
for fp in file_paths:
    print(f"Lade Datei: {fp}")
    df = pd.read_json(fp, orient="records")
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)
print(f"Gesamtanzahl Posts: {len(df_all)}")

# Spalte 'date' vereinheitlichen
df_all['date'] = df_all['date'].astype(str)

# Parsing des Datums
def smart_parse(val):
    s = str(val)
    # bekannte 1970-Platzhalter ignorieren
    if s.startswith('1970-01-01'):
        return pd.NaT
    try:
        i = int(s)
        if i < 10**13:
            return pd.to_datetime(i, unit='ms', errors='coerce')
        elif i < 10**16:
            return pd.to_datetime(i, unit='us', errors='coerce')
        else:
            return pd.NaT
    except:
        return pd.to_datetime(s, errors='coerce')

df_all['date_parsed'] = df_all['date'].map(smart_parse)


# Maske für Truth Social NaT-Werte
mask_ts_nat = df_all['date_parsed'].isna()
print(f"Truth Social NaT-Werte: {mask_ts_nat.sum()}")


# Versuch: Rohzahlen in 'date' reparieren
dates_raw = df_all.loc[mask_ts_nat, 'date'].str.extract(r'(\d{12,19})')[0]

# Millisekunden
parsed_ms = pd.to_datetime(dates_raw.astype(float), unit='ms', errors='coerce')
# Mikrosekunden
parsed_us = pd.to_datetime(dates_raw.astype(float), unit='us', errors='coerce')

# Datumsspalten (aufgeteilt in Date, Time, Day, Month, Year)
df_all['date'] = df_all['date_parsed'].dt.date.astype(str)
df_all['time'] = df_all['date_parsed'].dt.time.astype(str)
df_all['day'] = df_all['date_parsed'].dt.day
df_all['month'] = df_all['date_parsed'].dt.month
df_all['year'] = df_all['date_parsed'].dt.year

# Hilfsspalte entfernen
df_all.drop(columns=['date_parsed'], inplace=True)
# richtige Reihenfolge
key_order = ['id','text','date','time','day','month','year'] + \
            [col for col in df_all.columns if col not in ['id','text','date','time','day','month','year']]
data_ordered = df_all[key_order].to_dict(orient="records")

with open("tta_full.json", "w", encoding="utf-8") as f:
    json.dump(data_ordered, f, ensure_ascii=False, indent=2)

print("Fertig: 'tta_full.json' gespeichert")
print(df_all['year'].value_counts().sort_index())

Lade Datei: TTA/januarmarch24.json
Lade Datei: TTA/marchmai24.json
Lade Datei: TTA/juliseptember24.json
Lade Datei: TTA/septembernovember23.json
Lade Datei: TTA/apriljuli23.json
Lade Datei: TTA/novemberjanuar24.json
Lade Datei: TTA/spetemberdezember24.json
Lade Datei: TTA/juliseptember23.json
Lade Datei: TTA/oktoberjanuar22.json
Lade Datei: TTA/januarapril23.json
Lade Datei: TTA/junioktober22.json
Lade Datei: TTA/maijuli24.json
Lade Datei: TTA/tweets_01-08-2021.json
Lade Datei: TTA/januar21juni22.json
Gesamtanzahl Posts: 80358
Truth Social NaT-Werte: 0
Fertig: 'tta_full.json' gespeichert
year
2009       56
2010      142
2011      773
2012     3531
2013     8144
2014     5784
2015     7536
2016     4225
2017     2602
2018     3568
2019     7818
2020    12236
2021      158
2022     3927
2023     9919
2024     9939
Name: count, dtype: int64


In [103]:
## Ist in den Spalten "isFlagged", "isDeleted" und "device" Wert immer null?
df = pd.read_json("tta_full.json", orient="records")
cols_to_check = ["isFlagged", "isDeleted", "device"]
for col in cols_to_check:
    if col in df.columns:
        null_count = df[col].isna().sum()
        total = len(df)
        print(f"Spalte '{col}': {null_count}/{total} null Werte")
    else:
        print(f"Spalte '{col}' existiert nicht in der Datei.")
print(df["isDeleted"].value_counts(dropna=False))

Spalte 'isFlagged': 23787/80358 null Werte
Spalte 'isDeleted': 23787/80358 null Werte
Spalte 'device': 23787/80358 null Werte
isDeleted
f       55479
None    23787
t        1092
Name: count, dtype: int64


#### Jetzt sollen "factbase_posts_clean.json" und "tta_full.json" zusammengeführt werden:

In [2]:
import pandas as pd
import json

# Json Datei
cols_factbase = ["id", "platform", "date", "time", "day", "month", "year", "text"]
df_factbase = pd.read_json("factbase_posts_clean.json", orient="records")
df_factbase = df_factbase[cols_factbase]

df_tta = pd.read_json("tta_full.json", orient="records")

# Einheitliche Spalten
cols_order = [
    "id", "text", "date", "time", "day", "month", "year",
    "favorites", "retweets", "isRetweet", "isDeleted", "device", "isFlagged", "platform"
]

for col in cols_order:
    if col not in df_factbase.columns:
        df_factbase[col] = pd.NA
    if col not in df_tta.columns:
        df_tta[col] = pd.NA

# Kombinieren der Dataframes
df_combined = pd.concat([df_factbase, df_tta], ignore_index=True)
df_combined = df_combined[cols_order]

# fehlende Daten auffüllen
mask_needs_parsing = df_combined['date'].isna() | df_combined['date'].apply(lambda x: not isinstance(x, pd.Timestamp))
df_combined.loc[mask_needs_parsing, 'date'] = pd.to_datetime(
    df_combined.loc[mask_needs_parsing, 'date'], errors='coerce'
)

# Jahr, Monat und Tag aus Date ableiten
df_combined['year'] = df_combined['date'].dt.year.astype('Int64')
df_combined['month'] = df_combined['date'].dt.month.astype('Int64').astype(str).str.zfill(2)
df_combined['day'] = df_combined['date'].dt.day.astype('Int64').astype(str).str.zfill(2)

# Plattform ergänzen
mask_empty = df_combined['platform'].isna()
df_combined.loc[mask_empty & (df_combined['date'] <= "2021-01-08"), 'platform'] = "Twitter"
df_combined.loc[mask_empty & (df_combined['date'] > "2021-01-08") & (df_combined['date'] < "2022-02-01"), 'platform'] = "Suspended"
df_combined.loc[mask_empty & (df_combined['date'] >= "2022-02-01"), 'platform'] = "Truth Social"

# isDeleted
def update_is_deleted(row):
    if pd.notna(row['isDeleted']) and row['isDeleted'] not in [None, "", "nan"]:
        return row['isDeleted']
    if isinstance(row['platform'], str) and "Deleted" in row['platform']:
        return "t"
    return "f"

df_combined['isDeleted'] = df_combined.apply(update_is_deleted, axis=1)

df_combined = df_combined.where(pd.notna(df_combined), None)
# Nach Datum sortieren, damit die Reihenfolge stimmt
df_combined = df_combined.sort_values(by='date').reset_index(drop=True)

# Export
df_combined.to_json("tta_final.json", orient="records", force_ascii=False, indent=2, date_format="iso")
df_combined.to_csv("tta_final.csv", index=False, encoding="utf-8")

# Ergebnisse
print(df_combined['year'].value_counts().sort_index())
print(df_combined[["id", "date", "day", "platform", "isDeleted"]].head())

2009       56
2010      142
2011      773
2012     3531
2013     8144
2014     5784
2015     7536
2016     4225
2017     2602
2018     3568
2019     7818
2020    12236
2021      158
2022     3927
2023     9919
2024    10708
2025     4421
Name: year, dtype: Int64
           id       date day platform isDeleted
0  1698308935 2009-05-04  04  Twitter         f
1  1701461182 2009-05-05  05  Twitter         f
2  1737479987 2009-05-08  08  Twitter         f
3  1741160716 2009-05-08  08  Twitter         f
4  1773561338 2009-05-12  12  Twitter         f


In [125]:
print(df['date'].isna().sum())

0


In [126]:
df_combined.shape

(85548, 14)

In [127]:
df_combined.tail()

,id,text,date,time,day,month,year,favorites,retweets,isRetweet,isDeleted,device,isFlagged,platform
85543,7164,None,2025-08-25,12:41,25,08,2025,None,None,None,f,None,None,Truth Social
85544,7165,https://www.youtube.com/live/rlPfxyCnxHg?si=J5...,2025-08-25,11:33,25,08,2025,None,None,None,f,None,None,Truth Social
85545,7166,WHAT IS GOING ON IN SOUTH KOREA? Seems like a ...,2025-08-25,09:20,25,08,2025,None,None,None,f,None,None,Truth Social
85546,7167,"I PAID ZERO FOR INTEL, IT IS WORTH APPROXIMATE...",2025-08-25,09:14,25,08,2025,None,None,None,f,None,None,Truth Social
85547,7168,The incompetent Mayor of Chicago just stated t...,2025-08-25,08:42,25,08,2025,None,None,None,f,None,None,Truth Social


### Weitere Vorverabeitung der Daten
..damit diese brauchbare Daten für eine gute Analyse darstellen.

In [128]:
# Wenn bei der Spalte "Platform" "deleted" an erster Stelle steht, 
# dann sollte in der Spalte "isDeleted" "True" stehen.
df_deleted_platform = df_combined[df_combined["platform"].astype(str).str.contains("Deleted", na=False)]
print(df_deleted_platform[["id", "date", "platform", "isDeleted"]].head(20).T)

                             80361                    80409  \
id                            5814                     5764   
date           2024-11-03 00:00:00      2024-11-04 00:00:00   
platform   Deleted •  Truth Social  Deleted •  Truth Social   
isDeleted                        t                        t   

                             80439                    80542  \
id                            5768                     5671   
date           2024-11-04 00:00:00      2024-11-10 00:00:00   
platform   Deleted •  Truth Social  Deleted •  Truth Social   
isDeleted                        t                        t   

                             80599                    80674  \
id                            5610                     5540   
date           2024-11-14 00:00:00      2024-11-23 00:00:00   
platform   Deleted •  Truth Social  Deleted •  Truth Social   
isDeleted                        t                        t   

                             80748                  

In [129]:
count_without_time = df_combined["time"].isna().sum()
print(f"Anzahl der Posts ohne Zeitangabe: {count_without_time}")

Anzahl der Posts ohne Zeitangabe: 0


In [130]:
## Html-tags entfernen

In [131]:
import json
import re

with open("tta_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Regex für alle HTML-Tags und target-Attribute
TAG_RE = re.compile(r"<[^>]+>")
TARGET_RE = re.compile(r'target="[^"]*"')

def clean_html(raw_text):
    if not isinstance(raw_text, str):
        return raw_text
    # target entfernen
    no_target = TARGET_RE.sub("", raw_text)
    # alle Tags entfernen
    clean_text = TAG_RE.sub("", no_target)
    return clean_text.strip()

# Rekursive Bereinigung für Dicts, Listen, Strings
def recursive_clean(obj):
    if isinstance(obj, dict):
        return {k: recursive_clean(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [recursive_clean(i) for i in obj]
    elif isinstance(obj, str):
        return clean_html(obj)
    else:
        return obj

cleaned_data = recursive_clean(data)
with open("tta_final_clean.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, ensure_ascii=False, indent=2)

In [132]:
# hier die ordentlichen Daten ohne HTML-tags
import pandas as pd
tta_full = pd.read_json("tta_final_clean.json", orient="records")
print(tta_full.text.tail(15))

85533    Did Wes Moore, the Governor of Maryland, lie a...
85534    Despite a very high popularity and, according ...
85535    Governor Wes Moore of Maryland has asked, in a...
85536                                                 None
85537    The Appellate Court removed incompetent Judge ...
85538    https://humanevents.com/2025/08/21/shea-bradle...
85539    Why is it that ABC and NBC FAKE NEWS, two of t...
85540    I played Golf yesterday with the Great Roger C...
85541    https://www.whitehouse.gov/presidential-action...
85542    https://www.whitehouse.gov/presidential-action...
85543                                                 None
85544    https://www.youtube.com/live/rlPfxyCnxHg?si=J5...
85545    WHAT IS GOING ON IN SOUTH KOREA? Seems like a ...
85546    I PAID ZERO FOR INTEL, IT IS WORTH APPROXIMATE...
85547    The incompetent Mayor of Chicago just stated t...
Name: text, dtype: object


In [133]:
# und hier zum Vergleich nochmal die vorherigen Daten
import pandas as pd
tta_full = pd.read_json("tta_final.json", orient="records")
print(tta_full.text.tail(15).T)

85533    Did Wes Moore, the Governor of Maryland, lie a...
85534    Despite a very high popularity and, according ...
85535    Governor Wes Moore of Maryland has asked, in a...
85536                                                 None
85537    The Appellate Court removed incompetent Judge ...
85538    https://humanevents.com/2025/08/21/shea-bradle...
85539    Why is it that ABC and NBC FAKE NEWS, two of t...
85540    I played Golf yesterday with the Great Roger C...
85541    https://www.whitehouse.gov/presidential-action...
85542    https://www.whitehouse.gov/presidential-action...
85543                                                 None
85544    https://www.youtube.com/live/rlPfxyCnxHg?si=J5...
85545    WHAT IS GOING ON IN SOUTH KOREA? Seems like a ...
85546    I PAID ZERO FOR INTEL, IT IS WORTH APPROXIMATE...
85547    The incompetent Mayor of Chicago just stated t...
Name: text, dtype: object


In [134]:
# if text starts with("RT"): isRetweet = True
import pandas as pd
df = pd.read_json("tta_final_clean.json", dtype=str)

# 'isRetweet' für Tweets, die mit "RT" beginnen, setzen (nur wenn leer/NaN)
def fix_is_retweet(row):
    val = row.get("isRetweet")
    text = row.get("text", "")
    if val not in ["True", "False"]:
        if isinstance(text, str) and text.startswith("RT"):
            return "True"
        else:
            return "False"
    return val

df["isRetweet"] = df.apply(fix_is_retweet, axis=1)
df.to_json("tta_final_clean.json", orient="records", force_ascii=False, indent=2)

In [135]:
# Zeilen filtern, bei denen text mit "RT" beginnt
rt_rows = df[df["text"].str.startswith("RT", na=False)]
print(rt_rows[["text", "isRetweet"]])

                                                    text isRetweet
805    RT @Newsmax_Media: Trumps Warns of Obama Tippi...      True
1256   RT: @thedailybeast: Polling shows the @America...      True
1273   RT @accesshollywood: @realDonald Trump: 'Celeb...      True
1313   RT @OCChoppers: Bike we built for @realDonaldT...      True
2413   RT @IBDeditorials: Was Barack Obama A Foreign ...      True
...                                                  ...       ...
85422  RT: https://truthsocial.com/users/realDonaldTr...      True
85429  RT: https://truthsocial.com/users/realDonaldTr...      True
85435  RT: https://truthsocial.com/users/realDonaldTr...      True
85491  RT: https://truthsocial.com/users/realDonaldTr...      True
85511  RT: https://truthsocial.com/users/realDonaldTr...      True

[16383 rows x 2 columns]


doppelte IDs aufgrund der Zusammenführung von tta und den gescrapten Daten:

In [136]:
import json
from collections import Counter

with open("tta_final_clean.json", "r", encoding="utf-8") as f:
    data = json.load(f)
# Einträge ohne ID
missing_id_entries = [item for item in data if "id" not in item or item["id"] is None]
print(f"Anzahl der Einträge ohne ID: {len(missing_id_entries)}")

# die fehlenden Einträge anzeigen
for entry in missing_id_entries:
    print(entry)
    
## ids null-padden
ids = [str(item["id"]).zfill(6) for item in data]

# Doppelte IDs
counter = Counter(ids)
duplicates = [id_ for id_, count in counter.items() if count > 1]

print("\nDoppelte IDs:", duplicates)
print("\nAnzahl doppelter IDs:", len(duplicates))

Anzahl der Einträge ohne ID: 0

Doppelte IDs: ['1291488374835097600', '109136277636319040']

Anzahl doppelter IDs: 2


In [137]:
import json
with open("tta_final_clean.json", "r", encoding="utf-8") as f:
    data = json.load(f)

data_sorted = sorted(data, key=lambda x: x.get("date", ""))  

# neue Ids
num_entries = len(data_sorted)
id_width = len(str(num_entries)) # wie viele Stellen haben die IDs?
for i, item in enumerate(data_sorted, start=1):
    item["id"] = str(i).zfill(id_width)

with open("tta_final_clean_new.json", "w", encoding="utf-8") as f:
    json.dump(data_sorted, f, ensure_ascii=False, indent=4)

print(f"{len(data_sorted)} Einträge wurden neu nummeriert. Die erste ID ist {data_sorted[0]['id']}.")

85548 Einträge wurden neu nummeriert. Die erste ID ist 00001.


#### -> tta_final_clean_new.json ist das finale Dokument.

In [138]:
df.shape

(85548, 14)

In [139]:
df.columns

Index(['id', 'text', 'date', 'time', 'day', 'month', 'year', 'favorites',
       'retweets', 'isRetweet', 'isDeleted', 'device', 'isFlagged',
       'platform'],
      dtype='object')

In [1]:
import pandas as pd
df = pd.read_json("tta_final_clean_new.json", dtype=str)
print(df['year'].value_counts().sort_index())

2009       56
2010      142
2011      773
2012     3531
2013     8144
2014     5784
2015     7536
2016     4225
2017     2602
2018     3568
2019     7818
2020    12236
2021      158
2022     3927
2023     9919
2024    10708
2025     4421
Name: year, dtype: int64
